https://minimin2.tistory.com/137

https://leo-bb.tistory.com/62?category=858291

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.std.html

# 결측치 많은 변수들

'user_test_correct_cnt_per_test': 372603,

'user_acc_per_test': 372603,

'user_total_acc': 365164,

 'user_total_correct_cnt': 365164

In [1]:
# !pip install pycaret > /dev/null

In [1]:
!pip install pycaret[full] ngboost


  Using cached shap-0.39.0.tar.gz (356 kB)
  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-omryoqjp/shap/setup.py'"'"'; __file__='"'"'/tmp/pip-install-omryoqjp/shap/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-n7ep_cfz
       cwd: /tmp/pip-install-omryoqjp/shap/
  Complete output (119 lines):
  The nvcc binary could not be located in your $PATH. Either  add it to your path, or set $CUDAHOME to enable CUDA
  Error building cuda module: TypeError('cannot unpack non-iterable NoneType object')
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.7
  creating build/lib.linux-x86_64-3.7/shap
  copying shap/__init__.py -> build/lib.linux-x86_64-3.7/shap
  copying shap/_

# LGBM with pycaret



In [2]:
import numpy as np
import pandas as pd
import os
import random
import pickle
from pycaret.classification import *
from pycaret.utils import check_metric
import time
from datetime import timedelta, timezone, datetime
from copy import deepcopy
from collections import defaultdict
from tqdm import tqdm_notebook, tqdm
import json

# 데이터 가져오기

In [3]:
upper_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset'
train_path = f'{upper_dir}/train_data.csv'
test_path = f'{upper_dir}/test_data.csv'
submission_path = f'{upper_dir}/sample_submission.csv'

In [4]:
train = pd.read_csv(train_path) 
test = pd.read_csv(test_path)

answerCode2bool = {'userID':object,  'answerCode': 'int16', 'KnowledgeTag':object}
train = train.astype(answerCode2bool)
test = test.astype(answerCode2bool)
# train.Timestamp = pd.to_datetime(train.Timestamp)
# test.Timestamp = pd.to_datetime(test.Timestamp)

# Feature Engineering

## Train에서 미리 추출해야할 변수들

In [5]:
# # trian에서 각 문제별 통계량 추출
# # 아래의 피쳐들은 train에서 뽑은 값들을 test에 대입해주어야함.

# testId_mean_sum_count = train.groupby(['testId'])['answerCode'].agg(['mean','sum','count']).to_dict()
# assessmentItemID_mean_sum_count = train.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
# KnowledgeTag_mean_sum_count = train.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','count']).to_dict()

total_num_prob_in_test = train[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()  
testid_maxlen_dict = total_num_prob_in_test.to_dict()

# LB Prediction해야하는 문항 ID들을 파악
set_assessmentItemID = set(test.loc[test.answerCode == -1, 'assessmentItemID'].values)
set_testID = set(test.loc[test.answerCode == -1, 'testId'].values)

## Input Data에 대해서 만들어줘야 하는 피쳐들

In [6]:
def minyong_feature(df):
    df['test_pre'] = df.testId.map(lambda x: int(x[1:4]))
    df['test_post'] = df.testId.map(lambda x: int(x[-3:]))
    return df

In [7]:
def ara_featunre(df):
    # ------- user의 평균 문제 풀이 시간 (시험지 별 체크)
    next_df = df.shift(-1).fillna(-1)
    prev_df = df.shift(1).fillna(-1)
    df["diff"] = pd.to_datetime(next_df["Timestamp"]) - pd.to_datetime(df["Timestamp"])

    pr_user_id = df["userID"][0]
    pr_test_num, diff = 1, 0
    user_current_test = defaultdict(int)
    user_ox_count = dict()
    user_continue_ox_count = dict()
    user_assessmentnum_answerrate = dict()

    user_assessment_avg, user_test_done_time = [], []
    user_answer_cnt, user_wrong_cnt = [], []
    user_continue_answer_cnt, user_continue_wrong_cnt = [], []
    user_test_answer_rate = []
    i=-1
    for user_id, pr_test, pr_assess, nx_test, nx_assess, diff_time, prev_answer in tqdm(zip(df["userID"], df["testId"], df["assessmentItemID"], next_df["testId"], next_df["assessmentItemID"], df["diff"], prev_df["answerCode"])):
        i+=1

        if user_id != pr_user_id: # 다른 user 시작
            pr_test_num, diff = 1, 0
            user_current_test = defaultdict(int)
            user_ox_count = dict()
            user_continue_ox_count = dict()
            user_assessmentnum_answerrate = dict()
            pr_user_id = user_id
            prev_answer = -1
        
        if user_current_test[pr_test] == 0 or nx_test == -1: # 새로운 시험지거나 맨 마지막일 경우
            prev_answer = -1
        
        if pr_test not in user_current_test:
            user_current_test[pr_test] = 1 # 현재 시험지의 푼 문항 수 체크
        else:
            user_current_test[pr_test] += 1
        
        if pr_test not in user_ox_count:
            prev_count = [1, 0] if prev_answer == 1 else [0, 1] # Answer / Wrong count
            prev_num_rate = [1, 1, 1.0] if prev_answer == 1 else [1, 0, 0.0] # 푼 문제 수, 현재까지 정답 수, 정답률
            if prev_answer == -1:
                prev_count = [0, 0]
            
            user_ox_count[pr_test] = deepcopy(prev_count)
            user_continue_ox_count[pr_test] = deepcopy(prev_count)
            user_assessmentnum_answerrate[pr_test] = deepcopy(prev_num_rate) # 시험지 별 푼 문항 수, 정답률
        else:
            if prev_answer == 1:
                user_ox_count[pr_test][0] += 1
                user_continue_ox_count[pr_test][0] += 1 # 이전 답이 1 이라면 +1
                user_continue_ox_count[pr_test][1] = 0  # 연속 오답 count는 초기화
                user_assessmentnum_answerrate[pr_test][0] += 1 # 푼 문항 수 증가
                user_assessmentnum_answerrate[pr_test][1] += 1 # 푼 문항 중 정답수 증가
        
            else:
                user_ox_count[pr_test][1] += 1
                user_continue_ox_count[pr_test][0] = 0  # 연속정답 count는 초기화
                user_continue_ox_count[pr_test][1] += 1 # 이전 답이 0 이라면 +1
                user_assessmentnum_answerrate[pr_test][0] += 1 # 푼 문항 수 증가
        
        user_assessmentnum_answerrate[pr_test][2] = user_assessmentnum_answerrate[pr_test][1]/user_assessmentnum_answerrate[pr_test][0] # 정답률 갱신

        user_answer_cnt.append(user_ox_count[pr_test][0])
        user_wrong_cnt.append(user_ox_count[pr_test][1])

        user_continue_answer_cnt.append(user_continue_ox_count[pr_test][0])
        user_continue_wrong_cnt.append(user_continue_ox_count[pr_test][1])

        user_test_answer_rate.append(user_assessmentnum_answerrate[pr_test][2])


        if user_current_test[pr_test] == testid_maxlen_dict[pr_test]: # 시험지의 문제를 모두 다 풀었다면
            del user_current_test[pr_test] # 풀고있는 시험지에서 제외 (모두 다 풀었음)
            del user_ox_count[pr_test]
            del user_continue_ox_count[pr_test]
            del user_assessmentnum_answerrate[pr_test]

        if len(user_current_test) == 0 or nx_test == -1: # 만약 usre가 시작한 시험지를 모두 푸는데 완료했다면
            user_test_done_time.append(diff + user_assessment_avg[-1]) # 마지막 문제는 현재 걸린 시간에 평균적으로 걸린 시간만큼만 더해줌
            user_assessment_avg.append(user_assessment_avg[-1]) # 마지막 문제는 이전의 평균 값으로 사용

            pr_test_num, diff = 1, 0
            user_current_test = defaultdict(int)
            user_ox_count = dict()
            user_continue_ox_count = dict()
            user_assessmentnum_answerrate = dict()
        
        else: # 하나라도 끝내지 못한 시험지가 남아있다면
            diff += diff_time.total_seconds()
            user_test_done_time.append(diff)
            user_assessment_avg.append(diff/pr_test_num)
        
        pr_test_num += 1
    
    df = pd_join(df, "userSolTime", user_assessment_avg) # user의 문항을 푸는데 마친 평균 시간
    df = pd_join(df, "userTestTime", user_test_done_time) # user의 시험지 별 푸는데 마친 시간
    df = pd_join(df, "userTestAnswer", user_answer_cnt) # user의 시험지 별 정답 수
    df = pd_join(df, "userTestWrong", user_wrong_cnt) # user의 시험지 별 오답 수
    df = pd_join(df, "userTestContAnswer", user_continue_answer_cnt) # user의 시험지 별 연속 정답 수
    df = pd_join(df, "userTestContWrong", user_continue_wrong_cnt) # user의 시험지 별 연속 오답 수
    df = pd_join(df, "userTestCorrectRate", user_test_answer_rate) # user의 시험지 별 정답률

    # 유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['correctAnswer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['correctAnswer'] = df['correctAnswer'].fillna(0)
    df['totalAnswer'] = df.groupby('userID')['answerCode'].cumcount()
    df['userAcc'] = df['correctAnswer']/df['totalAnswer']
    df['userAcc'] = df['userAcc'].fillna(0)
    return df

# 결측치 채우기

In [8]:
# 아라 feature 2개 결측치 채우기
def ara_fillna(df):
    df['userAcc'] = df['userAcc'].fillna(0)
    df['correctAnswer'] = df['correctAnswer'].fillna(0)
    return df

# 내꺼 feature 4개 결측치 채우기
def mjh_fillna(df):
    df['user_total_correct_cnt'] = df['user_total_correct_cnt'].fillna(0)
    df['user_total_acc'] = df['user_total_acc'].fillna(0)
    df['user_test_correct_cnt_per_test'] = df['user_test_correct_cnt_per_test'].fillna(0)
    df['user_acc_per_test'] = df['user_acc_per_test'].fillna(0)
    return df

In [9]:
asset_dir = '/opt/p4-dkt-freshtomato/input/data/train_dataset/dict_for_feature'
# LB prediction에서 맞춰야하는 문항 ID 파악
def pd_join(data, column_name, value):
    value = pd.DataFrame(value, columns=[f"{column_name}"])
    data = data.join(value)
    return data
def save_pickle(path, values):
    os.makedirs(asset_dir, exist_ok=True)
    with open(f"{asset_dir}/{path}", 'wb') as f:
        pickle.dump(values, f)

def load_pickle(path):
    with open(f"{asset_dir}/{path}", 'rb') as f:
        load_values = pickle.load(f)
    return load_values


# Feature Engineering

In [10]:
minyoung = False

ara = False
ara_fillna_bool = False

mjh_fillna_bool = False

In [11]:
def feature_engineering(df, is_train):
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################

    if is_train:
        # 본 피쳐들은 train에서 얻어진 값을 그대로 유지합니다.
        testId_mean_sum_count = df.groupby(['testId'])['answerCode'].agg(['mean','sum','count']).to_dict()
        save_pickle("testId_mean_sum_count.pk",testId_mean_sum_count)

        assessmentItemID_mean_sum_count = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
        save_pickle("assessmentItemID_mean_sum_count.pk",assessmentItemID_mean_sum_count)

        KnowledgeTag_mean_sum_count = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum','count']).to_dict()
        save_pickle("KnowledgeTag_mean_sum_count.pk",KnowledgeTag_mean_sum_count)

    # -> inference할 때는, 저장된 정답률을 가져와서 mapping하는 코드입니다.
    testId_mean_sum_count = load_pickle("testId_mean_sum_count.pk")
    df["test_mean"] = df.testId.map(testId_mean_sum_count['mean'])
    df['test_sum'] = df.testId.map(testId_mean_sum_count['sum'])

    assessmentItemID_mean_sum_count = load_pickle("assessmentItemID_mean_sum_count.pk")
    df["ItemID_mean"] = df.assessmentItemID.map(assessmentItemID_mean_sum_count['mean'])
    df['ItemID_sum'] = df.assessmentItemID.map(assessmentItemID_mean_sum_count['sum'])

    KnowledgeTag_mean_sum_count = load_pickle("KnowledgeTag_mean_sum_count.pk")
    df["tag_mean"] = df.KnowledgeTag.map(KnowledgeTag_mean_sum_count['mean'])
    df['tag_sum'] = df.KnowledgeTag.map(KnowledgeTag_mean_sum_count['sum'])

    print("1번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################

    ## 문제번호 (시험지 내에서) (범주형? 연속형?) - 3자리
    # 0 부터 시작하도록 
    df['problem_number'] = df.assessmentItemID.map(lambda x: int(x[-3:])-1)
    if minyoung ==True:
        df = minyong_feature(df)

    ## 시험지의(testID 별) 문제 최대개수가 몇인지 dict
    ## (이를 파악하여 중복 풀이하는 친구들을 걸러낼수있음)
    ## 0.37초 <- 아라가 짜준 코드는 1.4초
    total_num_prob_in_test = df[['assessmentItemID', 'testId']].drop_duplicates().groupby('testId').size()  
    testid_maxlen_dict = total_num_prob_in_test.to_dict()
    print("2번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    # 아라가 만들어준 피쳐들 추가
    if ara==True:
        df = ara_featunre(df)
    if ara_fillna_bool==True:
        df = ara_fillna(df)
    print("(3-1)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df.Timestamp = pd.to_datetime(df.Timestamp)
    print("(3-2)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    # 시험지별 문제번호의 최대값
    # 0.877초
    max_prob_in_test = df.groupby('testId').problem_number.max()
    # (시험지별 문제개수 != 시험지별 문제번호 최댓값)에 해당하는 시험지의 index를 추출한다. (양쪽다 Series를 이용)
    inconsistent_index = max_prob_in_test[max_prob_in_test +1 != total_num_prob_in_test].index  # max_prob_in_test는 0부터 시작하고, num_prob_in_test는 1부터 시작
    # 문제 max번호와 문제개수가 일치하지 않는 시험지에 해당하는 데이터프레임으로부터
    # (시험지 Id와 문제 Id) 추출 후 문제 Id 기준 오름차순 정렬
    inconsistent_df = df.loc[df.testId.isin(inconsistent_index),['assessmentItemID', 'testId']].drop_duplicates().sort_values('assessmentItemID')

    # 순서대로 안 푼 시험지에서의 본래 문제의 순서가 dict에 저장된다 (inconsistent_Itemid_item_dict)
    inconsistent_Itemid_item_dict = {}
    inconsistent_df_group = inconsistent_df.groupby('testId')
    # 순서대로 안 푼 시험지들에 대하여 for문
    for key in inconsistent_df_group.groups:
        for i, (k,_) in enumerate(inconsistent_df_group.get_group(key).values):
            inconsistent_Itemid_item_dict[k] = i

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################

    # origin_problem_order : 한 시험지 내에서 해당 문제 Id의 본래 순서 (중간에 비어있는 문제는 없는 문제로 생각하여 다시 순서를 매김)
    # ex) A080096003,A080096001,A080096005,A080096006,A080096007,A080096008,A080096002 
    # 문제번호 : 3, 1, 5, 6, 7, 8, 2 => 문제번호 중 4번이 빠져있음
    # 따라서 4번을 제외하고 문제번호를 본래순서에 대응시키면 => (1:0,2:1,3:2,5:3,6:4,7:5,8:6)
    # 변수생성 => origin_problem_order : (2,0,3,4,5,6,1)
    # 시험 문제번호의 최대값과 시험지 내의 문제개수가 일치하지 않는 시험지의 testId들을 추출        
    df['origin_problem_order'] =  df.assessmentItemID.map(lambda x: int(inconsistent_Itemid_item_dict[x]) if x in inconsistent_Itemid_item_dict else int(x[-3:]) - 1) # 0부터 시작하도록 -1 해줌.
    print("4번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    print("5번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    
    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    # 유저가 푼 시험지들에 대해, 유저의 전체 정답/풀이횟수/정답률 계산
    df_group = df.groupby(['userID','testId'])['answerCode']
    print("(6-0)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df['user_total_correct_cnt'] = df_group.transform(lambda x: x.cumsum().shift(1))    # 정답개수
    print("(6-1)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df['user_total_ans_cnt'] = df_group.cumcount()  # 풀이횟수
    print("(6-2)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    df['user_total_acc'] = df['user_total_correct_cnt'] / df['user_total_ans_cnt']  # 정답률
    print("(6-3)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    #############################################################################################
    start = time.time()  # 시작 시간 저장 
    #############################################################################################
    # 유저가 풀고 있는 시험지에 대해, 현재 몇번째 문제인지
    df['total_num_prob_in_test'] = df.testId.map(testid_maxlen_dict)    # 문제수
    # 특정 시험지를 몇번째 반복하여 푸는지 계산
    df['nth_test'] = df['user_total_ans_cnt'] // df['total_num_prob_in_test'] # (같은)시험 완료 횟수
    df['user_test_ans_cnt'] = df['user_total_ans_cnt'] % df['total_num_prob_in_test']   # 현재 몇번째 문제인지
    print("7번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    # 각 시험지 당 유저의 정확도를 계산
    start = time.time()  # 시작 시간 저장 
    df['user_test_correct_cnt_per_test'] = df.groupby(['userID','testId','nth_test'])['answerCode'].transform(lambda x: x.cumsum().shift(1))
    print("(8-1)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간
    start = time.time()  # 시작 시간 저장 
    df['user_acc_per_test'] = df['user_test_correct_cnt_per_test']/df['user_test_ans_cnt']
    print("(8-2)번째 Time Lapse :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

    df['int_KnowledgeTag'] = df.KnowledgeTag.astype('int64')
    df['test_total_id'] = df['assessmentItemID'].map(lambda x: int(x[1:7]))

    if mjh_fillna_bool==True:
        df = mjh_fillna(df)
    return df

In [12]:
df = feature_engineering(train, is_train=True)
df_test = feature_engineering(test, is_train=False)


1번째 Time Lapse : 3.8865859508514404
2번째 Time Lapse : 3.2219443321228027
(3-1)번째 Time Lapse : 2.384185791015625e-06
(3-2)번째 Time Lapse : 1.123673677444458
4번째 Time Lapse : 2.451723098754883
5번째 Time Lapse : 3.3691723346710205
(6-0)번째 Time Lapse : 0.0013575553894042969
(6-1)번째 Time Lapse : 170.81101369857788
(6-2)번째 Time Lapse : 0.10434675216674805
(6-3)번째 Time Lapse : 0.01326608657836914
7번째 Time Lapse : 0.371859073638916
(8-1)번째 Time Lapse : 169.14058685302734
(8-2)번째 Time Lapse : 0.014133214950561523
1번째 Time Lapse : 0.1909785270690918
2번째 Time Lapse : 0.23412656784057617
(3-1)번째 Time Lapse : 2.384185791015625e-06
(3-2)번째 Time Lapse : 0.07741808891296387
4번째 Time Lapse : 0.18863272666931152
5번째 Time Lapse : 0.18066716194152832
(6-0)번째 Time Lapse : 0.0006699562072753906
(6-1)번째 Time Lapse : 18.373064517974854
(6-2)번째 Time Lapse : 0.009275436401367188
(6-3)번째 Time Lapse : 0.003023386001586914
7번째 Time Lapse : 0.03955554962158203
(8-1)번째 Time Lapse : 21.282508850097656
(8-2)번째 Time Lapse

In [14]:
# def mjh_mean_fillna(df):
#     df['user_total_correct_cnt'] = df['user_total_correct_cnt'].fillna(df['user_total_correct_cnt'].mean())
#     df['user_total_acc'] = df['user_total_acc'].fillna(df['user_total_acc'].mean())
#     df['user_test_correct_cnt_per_test'] = df['user_test_correct_cnt_per_test'].fillna(df['user_test_correct_cnt_per_test'].mean())
#     df['user_acc_per_test'] = df['user_acc_per_test'].fillna(df['user_acc_per_test'].mean())
#     return df
# df = mjh_mean_fillna(df)
# df_test = mjh_mean_fillna(df_test)

In [13]:
print(f'df.shape : {df.shape}')
print(f"df_test.shape : {df_test.shape}")

df.shape : (2266586, 24)
df_test.shape : (260114, 24)


In [14]:
df_csv_name_after_FE = f'{upper_dir}/df_피쳐내꺼만_24개_결측치그대로놔둠.csv'
df.to_csv(df_csv_name_after_FE,index=False)
# df = pd.read_csv(df_csv_name_after_FE)
print(f"df_csv_name_after_FE : {df_csv_name_after_FE}")

df_csv_name_after_FE : /opt/p4-dkt-freshtomato/input/data/train_dataset/df_피쳐내꺼만_24개_결측치그대로놔둠.csv


In [23]:
df.shape

(2266586, 24)

In [24]:
df_test_shift = df_test[df_test['userID'] != df_test['userID'].shift(-1)]

## 엔지니어링 이후의 EDA

In [ ]:
# 생성된 Feature 잘못된거 없는지 확인
# c = 'test_mean'
# print(df.groupby(['userID']).apply(lambda x: x.iloc[-1])[c].isnull().sum())
# df.groupby(['userID']).apply(lambda x: x.iloc[-1])[c].value_counts()

# print(df_test.groupby(['userID']).apply(lambda x: x.iloc[-1])[c].isnull().sum())
# df_test.groupby(['userID']).apply(lambda x: x.iloc[-1])[c].value_counts()

num_nan_column_train = {}
num_nan_column_test = {}
from tqdm import tqdm_notebook
for c in tqdm_notebook(df.columns):
    nan_count = df.groupby(['userID']).apply(lambda x: x.iloc[-1])[c].isnull().sum()
    if nan_count!=0:
        num_nan_column_train[c] = nan_count
    nan_count = df_test.groupby(['userID']).apply(lambda x: x.iloc[-1])[c].isnull().sum()
    if nan_count!=0:
        num_nan_column_test[c] = nan_count

# 각 User별 마지막 행에 결측치가 있는지 조사하는 코드 (마지막 행 결측치 존재할 때, column이 출력됨)
print('User 별 마지막 row에 결측치가 존재하는 경우')
print(len(num_nan_column_train))
print(len(num_nan_column_test))
print(num_nan_column_train)
print(num_nan_column_test)

In [ ]:
num_nan_column_train = {}
num_nan_column_test = {}
from tqdm import tqdm_notebook
for c in tqdm_notebook(df.columns):
    nan_count = df.groupby(['userID']).apply(lambda x: x.iloc[0])[c].isnull().sum()
    if nan_count!=0:
        num_nan_column_train[c] = nan_count
    nan_count = df_test.groupby(['userID']).apply(lambda x: x.iloc[0])[c].isnull().sum()
    if nan_count!=0:
        num_nan_column_test[c] = nan_count

# 각 User별 첫 행에 결측치가 있는지 조사하는 코드 (첫 행 결측치 존재할 때, column이 출력됨)
print('User 별 첫 row에 결측치가 존재하는 경우')
print(len(num_nan_column_train))
print(len(num_nan_column_test))
print(num_nan_column_train)
print(num_nan_column_test)

In [26]:
# 모든 데이터셋에 대하여 결측치가 1개 이상인 column, 결측치 개수
{k:v for k,v in df.isnull().sum().to_dict().items() if v!=0}

{'user_total_correct_cnt': 365164,
 'user_total_acc': 365164,
 'user_test_correct_cnt_per_test': 372603,
 'user_acc_per_test': 372603}

In [16]:
# # 아라 feature 2개 결측치 채우기
# def ara_fillna(df):
#     df['userAcc'] = df['userAcc'].fillna(0)
#     df['correctAnswer'] = df['correctAnswer'].fillna(0)
#     return df

# # 내꺼 feature 4개 결측치 채우기
# def mjh_fillna(df):
#     df['user_total_correct_cnt'] = df['user_total_correct_cnt'].fillna(0)
#     df['user_total_acc'] = df['user_total_acc'].fillna(0)
#     df['user_test_correct_cnt_per_test'] = df['user_test_correct_cnt_per_test'].fillna(0)
#     df['user_acc_per_test'] = df['user_acc_per_test'].fillna(0)
#     return df

In [27]:
df.user_test_correct_cnt_per_test.value_counts()

1.0     507090
2.0     420585
3.0     331972
0.0     247793
4.0     221177
5.0      97507
6.0      44513
7.0      15374
8.0       4740
9.0       1896
10.0       891
11.0       392
12.0        53
Name: user_test_correct_cnt_per_test, dtype: int64

# Split Train valid Set

In [28]:
is_all_data = False
valid_only_LB = 'problem'   # '', 'test' # prediction에 사용할 test row들의 (문제ID, 시험지ID)에 해당하는 row만 추출하여 Valid한다.
train_except_holdout = False   # 'index', 'user' # holdout과 train을 겹치지 않게 할지, 그렇다면 그 기준은 무엇으로 할지
train_only_LB = False   # 'problem', 'test' # prediction에 사용할 test row들의 (문제ID, 시험지ID)에 해당하는 row만 추출하여 Train한다.
train_with_finalrow = 'test'  # None 'test','user' # 시험지 혹은 유저별 마지막 row만 가져와서 학습할 것인지
seed = 47


In [29]:
def train_holdout_split(
                        df, 
                        is_all_data=is_all_data,  # input df가 train과 test 모두인지,
                        valid_only_LB=valid_only_LB,
                        train_except_holdout=train_except_holdout,
                        train_with_finalrow = train_with_finalrow,
                        train_only_LB = train_only_LB,
                        ):

    # 모두 train으로 사용한다면 answerCode==-1에 해당하는 row는 모두 지워준다.
    if is_all_data:
        copy_df = df[df.answerCode != -1].copy()
    else:
        copy_df = df.copy()

    # 유저별 마지막 row에 해당하는 데이터만 Holdout으로 사용
    # 추가적으로 LB Test set과 동일 조건의 row만 Valid에서 추출할지 결정
    if valid_only_LB=='':
        holdout = df[(df['userID'] != df['userID'].shift(-1))].copy()
    # True라면, 추가조건으로 리더보드와 동일 문제ID에 해당하는 row만 추출하여 holdout로 사용
    elif valid_only_LB=='problem':
        holdout = df[(df['userID'] != df['userID'].shift(-1)) & (df.assessmentItemID.isin(set_assessmentItemID))].copy()
    # True라면, 추가조건으로 리더보드와 동일 시험ID에 해당하는 row만 추출하여 holdout로 사용
    elif valid_only_LB=='test':
        holdout = df[(df['userID'] != df['userID'].shift(-1)) & (df.testId.isin(set_testID))].copy()

    print(f"set(holdout.userID) : {len(set(holdout.userID))}")
    print(f"set(holdout.index) : {len(set(holdout.index))}")
    if train_except_holdout=='user':
        tr_val = copy_df[ copy_df['userID'].isin(set(holdout.userID)) == False ].copy()
    elif train_except_holdout=='index':
        tr_val = copy_df[ copy_df.index.isin(set(holdout.index)) == False ].copy()
    else:
        tr_val = copy_df.copy()
    print(f"(tr_val.shape) : {tr_val.shape}")

    # 학습시 무엇을 기준으로 마지막 row만 가져올지
    if train_with_finalrow == 'test':
        tr_val = tr_val[tr_val['testId'] != tr_val['testId'].shift(-1)].copy()
    elif train_with_finalrow == 'user':
        tr_val = tr_val[tr_val['userID'] != tr_val['userID'].shift(-1)].copy()
    else:
        pass

    # LB prediction의 row에서 맞혀야하는 문제들만 학습할 것인지 (444개)
    if train_only_LB=='problem':
        tr_val = tr_val[tr_val.assessmentItemID.isin(set_assessmentItemID)].copy()
    # LB prediction의 row에서 맞혀야하는 시험지들만 학습할 것인지 (411개)
    elif train_only_LB=='test':
        tr_val = tr_val[tr_val.testId.isin(set_testID)].copy()
    else:
        pass

    return tr_val, holdout

In [30]:
seed = 47
fold = 10
method = 'soft' # 'soft' , 'hard' : 모델들 blend할 때, 앙상블방법



tr_val, holdout = train_holdout_split(
                                    df, 
                                    is_all_data=is_all_data,
                                    valid_only_LB=valid_only_LB,
                                    train_except_holdout=train_except_holdout,
                                    train_with_finalrow = train_with_finalrow,
                                    train_only_LB = train_only_LB,
                                    )
print(len(tr_val), len(holdout))
print(tr_val.answerCode.mean(), holdout.answerCode.mean())


set(holdout.userID) : 4366
set(holdout.index) : 4366
(tr_val.shape) : (2266586, 24)
374594 4366
0.5601958386946935 0.48259276225377923


# AUTOML (NGBoosting도 쓸 수 있으니 참고)

In [31]:
## scikit-learn 기반 모델이면 다음과 같이 다 써볼수가 있음.
from ngboost import NGBClassifier
# ngc = NGBClassifier()
# ngboost = create_model(ngc)

model_name_list=["lightgbm"]   # lightgbm, gbc, lda, catboost, et, ada, rf, lr, nb, knn, qda, dt, xgboost, ngboost
ensemble_method = 'blend_models'  # 'blend_models', 'stack_models' # (Classifier : 'stack_models'), (Regression : 'create_stacknet')
meta_model_for_stack = 'xgboost' # If using stack_models

## 모델, grid score df, plot, config, output csv 파일들이 저장될 상위 경로

In [33]:
# 모델들, grid score df, plot 등이 모두 저장되는 상위경로
now_time = datetime.now(timezone(timedelta(hours=9))).strftime('%m%d_%H%M')
upper_dir_of_pycaret = f'/opt/p4-dkt-freshtomato/output_pycaret/{now_time}'
print(f"upper_dir_of_pycaret : {upper_dir_of_pycaret}")

upper_dir_of_pycaret : /opt/p4-dkt-freshtomato/output_pycaret/0603_0818


##  각 단계마다 (create - tune - ensemble - finalize) 모델, 정보 저장해주는 코드

In [34]:
def plot_save_pycaret_model(name, model,mode='tune'):
    # Feature Importance Plot 저장하기
    plot_dir = f'{upper_dir_of_pycaret}/plot_result'
    os.makedirs(plot_dir, exist_ok=True)
    os.chdir(plot_dir)
    # top10 feature
    feature_importance_plot = plot_model(model, plot = 'feature', save=True)
    os.rename(f'{plot_dir}/Feature Importance.png',f'{plot_dir}/{mode}_{name}_10_feature_importance.png')
    # all feature
    feature_importance_plot = plot_model(model, plot = 'feature_all', save=True)
    os.rename(f'{plot_dir}/Feature Importance (All).png',f'{plot_dir}/{mode}_{name}_all_feature_importance.png')

# def create_and_save_pycaret_model(name):
#     created_model = create_model(name, cross_validation = True)
#     # Display되는 score grid dataframe도 가져올 수가 있다. 
#     df_result_model = pull()
#     df_result_model.to_csv(f'{upper_dir_of_pycaret}/{name}_create_grid_df.csv')
#     # 실험결과를 저장하는 것도 가능하다.
#     save_experiment(f'{upper_dir_of_pycaret}/{name}_create_exp_result')
#     # 모델을 저장하는 것도 가능하다.
#     save_model(created_model, f'{upper_dir_of_pycaret}/{name}_model_saved')
#     return created_model

def tune_and_save_pycaret_model(name, model, optimize = 'AUC', fold = fold, n_iter = 10):
    tuned_model = tune_model(model, optimize = optimize, fold = fold, n_iter = n_iter)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_tuned_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    plot_save_pycaret_model(name, tuned_model, mode='tune')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_tuned_exp_result')
    except:
        print('tuned model의 실험결과는 저장할 수 없습니다.')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(tuned_model, f'{upper_dir_of_pycaret}/models/{name}_tuned_model_saved')
    return tuned_model

def ensemble_and_save_pycaret_model(
                                    model_name_list,
                                    estimator_list,
                                    fold = fold,
                                    optimize = 'AUC',
                                    method = method,
                                    meta_model = meta_model_for_stack,
                                    mode = ensemble_method,
                                    ):
    print('Now Ensemble the models....')
    if mode=='blend_models':
        ensembled_model = blend_models(estimator_list = estimator_list, fold = fold, optimize = optimize, method = method)
    elif mode=='stack_models':
        ensembled_model = stack_models(estimator_list = estimator_list, fold = fold, optimize = optimize, meta_model = meta_model)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    name = '_'.join(model_name_list)
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_ensembled_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    try:
        plot_save_pycaret_model(name, ensembled_model, mode='tune')
    except:
        print(f'ensemble model은 변수중요도 plot을 저장할 수 없습니다')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_ensembled_exp_result')
    except:
        print(f'ensemble model은 실험결과를 저장할 수 없습니다')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(ensembled_model, f'{upper_dir_of_pycaret}/models/{name}_ensembled_model_saved')
    return ensembled_model


def finalize_and_save_pycaret_model(model_name_list,model,):
    print('Now Finalizing the model....')
    finalized_model = finalize_model(model)
    # Display되는 score grid dataframe도 가져올 수가 있다. 
    df_result_model = pull()
    name = '_'.join(model_name_list)
    os.makedirs(f"{upper_dir_of_pycaret}/grid_df",exist_ok=True)
    df_result_model.to_csv(f'{upper_dir_of_pycaret}/grid_df/{name}_finalized_grid_df.csv')
    # 모델의 각종 plot을 저장하는 것도 가능하다.
    try:
        plot_save_pycaret_model(name, finalized_model, mode='tune')
    except:
        print(f'final model은 변수중요도 plot을 저장할 수 없습니다')
    # 실험결과를 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/expr",exist_ok=True)
    try:
        save_experiment(f'{upper_dir_of_pycaret}/expr/{name}_finalized_exp_result')
    except:
        print(f'final model은 실험결과를 저장할 수 없습니다')
    # 모델을 저장하는 것도 가능하다.
    os.makedirs(f"{upper_dir_of_pycaret}/models",exist_ok=True)
    save_model(finalized_model, f'{upper_dir_of_pycaret}/models/{name}_finalized_model_saved')
    return finalized_model


In [61]:
# setup(data=tr_val[FEATS], target='answerCode', train_size=0.8, categorical_features=cat_features, numeric_features=continuous_features)
# created_model = create_model('dt', cross_validation = True)
# df_results = pull()

# os.makedirs(upper_dir_of_pycaret,exist_ok=True)
# df_results.to_csv(f'{upper_dir_of_pycaret}/df_results.csv')
# pd.read_csv(f'{upper_dir_of_pycaret}/name_ensembled_grid_df.csv',index_col=0)


In [35]:
print(df.columns) # FEATS 참고용

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'test_mean', 'test_sum', 'ItemID_mean', 'ItemID_sum',
       'tag_mean', 'tag_sum', 'problem_number', 'origin_problem_order',
       'user_total_correct_cnt', 'user_total_ans_cnt', 'user_total_acc',
       'total_num_prob_in_test', 'nth_test', 'user_test_ans_cnt',
       'user_test_correct_cnt_per_test', 'user_acc_per_test',
       'int_KnowledgeTag', 'test_total_id'],
      dtype='object')


In [37]:
experiment_name = 'USEFEAT-mjh-only_NOT-FILLNA'
FEATS = [c for c in df.columns if c not in ['userID', 'assessmentItemID', 'testId', 'diff']]

cat_features = ['problem_number','origin_problem_order',
              'total_num_prob_in_test','nth_test',
              'user_test_ans_cnt','user_test_correct_cnt_per_test']

continuous_features = [f for f in FEATS if f not in cat_features+['answerCode','Timestamp','diff']]

# continuous_features += []

print(f'FEATS : {len(FEATS)}')
print(f"target : 'answerCode'")
print(f'cat_features : {len(cat_features)}')
print(f'continuous_features : {len(continuous_features)}')
print(f"etc_features : {[f for f in FEATS if f not in cat_features+continuous_features+['answerCode']]}")


FEATS : 21
target : 'answerCode'
cat_features : 6
continuous_features : 13
etc_features : ['Timestamp']


In [38]:
def ensemble_automl(
                    tr_val,
                    holdout,
                    FEATS,
                    cat_features=[],
                    continuous_features=[],
                    seed=seed,
                    model_name_list=model_name_list,
                    ensemble_method = ensemble_method,
                    meta_model_for_stack = meta_model_for_stack,
                    ):

    # tr_val, holdout = datasets
    random.seed(seed)
    settings = setup(
                    data=tr_val[FEATS], 
                    target='answerCode', 
                    train_size=0.8, 
                    categorical_features=cat_features, 
                    numeric_features=continuous_features,
                    # log_experiment=False,   # compare_model 할 때에는 하이퍼파라미터가 어떻게 튜닝되었는지 볼 수 있어서 유용하다고 한다.
                    experiment_name=experiment_name,
                    )

    # train_size만큼을 가지고 선언된 model_name_list 모델들을 학습을 함
    # create_model(name, sort='AUC', cross_validation = True)
    models_before_tune = [create_model(name, cross_validation = True) for name in model_name_list]
    # 앞서만든 모델들을 train_size만큼 가지고 튜닝함 (n_iter만큼 AutoML)
    # models_after_tune = [tune_model(model, optimize = 'AUC', fold = 10, n_iter = 10) for model in models_before_tune]
    models_after_tune = [tune_and_save_pycaret_model(name, model, optimize = 'AUC', fold = 10, n_iter = 10) for name, model in zip(model_name_list, models_before_tune)]


    # 튜닝된 모델들을 train_size만큼 가지고 앙상블
    if len(models_after_tune)<=1:
        ensembled = models_after_tune[0]
    else:
        ensembled = ensemble_and_save_pycaret_model(model_name_list,estimator_list=models_after_tune,fold = fold,optimize = 'AUC',method = method,meta_model = meta_model_for_stack,mode = ensemble_method)
    # elif ensemble_method=='blend_models':
    #     ensembled = blend_models(estimator_list = models_after_tune, fold = 10, method = 'soft', optimize = 'AUC')
    # elif ensemble_method=='stack_models':
    #     ensembled = stack_models(estimator_list = models_after_tune, meta_model = meta_model_for_stack, fold = 10, optimize = 'AUC')

    # 마지막 학습(Finalize)
    # 앞서 앙상블된 모델을 => setup으로 나눠져 쓰지않았던 valid까지 포함된 100퍼센트를 사용하여 fitting함
    final_model = finalize_and_save_pycaret_model(model_name_list,ensembled)

    metric_result = []
    prediction = predict_model(final_model, data=holdout[FEATS], raw_score = True)
    df_holdout_score = pull()

    os.makedirs(f"{upper_dir_of_pycaret}/holdout_score",exist_ok=True)
    df_holdout_score.to_csv(f'{upper_dir_of_pycaret}/holdout_score/finalize_holdout_score.csv')

    metric_result.append(f"HoldOut 데이터 ACC & AUC: {check_metric(prediction['answerCode'], prediction['Label'], metric = 'Accuracy')} ,{check_metric(prediction['answerCode'], prediction['Score_1'], metric = 'AUC')}")
    return final_model, metric_result

In [ ]:
final_model, metric_result = ensemble_automl(
                                            tr_val,
                                            holdout,
                                            FEATS,
                                            cat_features,
                                            continuous_features,
                                            seed=seed,
                                            model_name_list=model_name_list,
                                            ensemble_method = ensemble_method,
                                            meta_model_for_stack = meta_model_for_stack,
                                            )

print(final_model)

print(f"ACC & AUC : ",'\n'.join(metric_result))

final model은 실험결과를 저장할 수 없습니다
Transformation Pipeline and Model Succesfully Saved
LGBMClassifier(bagging_fraction=0.9, bagging_freq=2, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
               importance_type='split', learning_rate=0.3, max_depth=-1,
               min_child_samples=46, min_child_weight=0.001, min_split_gain=0.3,
               n_estimators=190, n_jobs=-1, num_leaves=20, objective=None,
               random_state=5914, reg_alpha=3, reg_lambda=0.0005, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
ACC & AUC :  HoldOut 데이터 ACC & AUC: 0.741 ,0.8198


In [40]:
num_category_per_feature = {}
for c in tqdm_notebook(df.columns):
    num_category_per_feature[c] = len(df[c].value_counts())
num_category_per_feature

  0%|          | 0/24 [00:00<?, ?it/s]

{'userID': 6698,
 'assessmentItemID': 9454,
 'testId': 1537,
 'answerCode': 2,
 'Timestamp': 2076774,
 'KnowledgeTag': 912,
 'test_mean': 1495,
 'test_sum': 850,
 'ItemID_mean': 5274,
 'ItemID_sum': 306,
 'tag_mean': 908,
 'tag_sum': 798,
 'problem_number': 13,
 'origin_problem_order': 13,
 'user_total_correct_cnt': 24,
 'user_total_ans_cnt': 27,
 'user_total_acc': 181,
 'total_num_prob_in_test': 12,
 'nth_test': 3,
 'user_test_ans_cnt': 13,
 'user_test_correct_cnt_per_test': 13,
 'user_acc_per_test': 47,
 'int_KnowledgeTag': 912,
 'test_total_id': 1537}



# Inference

In [41]:
config_key = [
            'experiment_name','is_all_data','valid_only_LB','train_except_holdout','train_only_LB','train_with_finalrow',
            'seed','fold','method','model_name_list','ensemble_method','meta_model_for_stack',
            'cat_features','continuous_features','FEATS'
            ]
config_value = [
                experiment_name,is_all_data,valid_only_LB,train_except_holdout,train_only_LB,train_with_finalrow,
                seed,fold,method,model_name_list,ensemble_method,meta_model_for_stack,
                cat_features,continuous_features,FEATS]


In [42]:
# MAKE PREDICTION
prediction = predict_model(final_model, data=df_test_shift[FEATS], raw_score=True)
total_preds = prediction.Score_1.values

# SAVE OUTPUT
prediction_name = f"피쳐{len(FEATS)}개사용_{experiment_name}" # 

output_dir = f'{upper_dir_of_pycaret}/submission'
os.makedirs(output_dir, exist_ok=True)    
write_path = os.path.join(output_dir, f"{prediction_name}.csv")
print("Saving Final Output Csv...")
with open(write_path, 'w', encoding='utf8') as w:
    print(f"writing prediction : {write_path}")
    w.write("id,prediction\n")
    for id, p in enumerate(total_preds):
        w.write('{},{}\n'.format(id,p))

# Save Config
write_path = os.path.join(output_dir, f"{prediction_name}_config.json")
config_dict = {k:v for k,v in zip(config_key, config_value)}
print("Saving Final Config Dict...")
## json파일 저장 ##
with open(write_path, "w") as fp:
    print(f"writing config : {write_path}")
    json.dump(config_dict, fp, indent=4)

Saving Final Output Csv...
writing prediction : /opt/p4-dkt-freshtomato/output_pycaret/0603_0818/submission/피쳐21개사용_USEFEAT-mjh-only_NOT-FILLNA.csv
Saving Final Config Dict...
writing config : /opt/p4-dkt-freshtomato/output_pycaret/0603_0818/submission/피쳐21개사용_USEFEAT-mjh-only_NOT-FILLNA_config.json


# 저장된 결과들 로드해오기

In [ ]:
# # load 해오는방법

## Loading Saved Model
# dt_saved = load_model('dt_saved_07032020')

# # Loading saved experiment
# experiment_saved = load_experiment('experiment_07032020')

# # Loading Saved grid score df
# name_tuned_grid_df = pd.read_csv(f'{upper_dir_of_pycaret}/name_tuned_grid_df.csv',index_col=0)

## get_logs function 연습해보기

It returns the logs for active experiment when log_experiment in setup is set as True. To retrieve logs for inactive experiment, use experiment_name parameter with get_logs. To save csv file locally, set save parameter as True.



In [108]:
from pycaret.datasets import get_data
data = get_data('diabetes')
# initializing setup
from pycaret.classification import *
clf1 = setup(data, target = 'Class variable', log_experiment = True, experiment_name = 'diabetes1')
# compare baseline
top5 = compare_models()
# checkout logs in csv
logs = get_logs(save=True) # To save csv in working directory

,Description,Value
0,session_id,559
1,Target,Class variable
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(768, 9)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:51:51
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Extreme Gradient Boosting


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7637,0.8415,0.6000,0.7117,0.6356,0.4653,0.4797,0.0530
lda,Linear Discriminant Analysis,0.7560,0.8087,0.5684,0.6978,0.6194,0.4439,0.4538,0.0090
lr,Logistic Regression,0.7542,0.8151,0.5684,0.6912,0.6193,0.4410,0.4489,0.3650
ridge,Ridge Classifier,0.7504,0.0000,0.5579,0.6900,0.6110,0.4311,0.4405,0.0080
rf,Random Forest Classifier,0.7469,0.8303,0.5263,0.7017,0.5922,0.4156,0.4305,0.3020
ada,Ada Boost Classifier,0.7468,0.7894,0.5579,0.6802,0.6080,0.4247,0.4326,0.0470
et,Extra Trees Classifier,0.7430,0.8076,0.5263,0.6930,0.5900,0.4092,0.4226,0.2810
dt,Decision Tree Classifier,0.7207,0.6971,0.6158,0.6084,0.6074,0.3916,0.3958,0.0080
knn,K Neighbors Classifier,0.7206,0.7409,0.5421,0.6205,0.5716,0.3690,0.3741,0.0720
nb,Naive Bayes,0.6666,0.7410,0.2211,0.6445,0.3144,0.1547,0.2033,0.0070


In [ ]:
logs